In [16]:
from feature.clip_make_dataset import make_data
from feature.after_dataset import accumulate
from feature.utils import log_scale
from model.model_train import model_train
import pandas as pd
from glob import glob
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import glob
import numpy as np
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [17]:
train_input_list = './data/train_input/*.csv'
train_target_list = './data/train_target/*.csv'

test_input_list = './data/test_input/*.csv'
test_target_list = './data/test_target/*.csv'
train, test, p_train, p_test = make_data(train_input_list, train_target_list, test_input_list, test_target_list)

In [18]:
accumulate(train, test, '누적분무량')
accumulate(train, test, '누적백색광량')
accumulate(train, test, '누적청색광량')
accumulate(train, test, '누적적색광량')

In [19]:
train_1 = pd.read_csv('./data/clip_preprocessing/clip_train_quantile0.25.csv')
test_1 = pd.read_csv('./data/clip_preprocessing/clip_test_quantile0.25.csv')
train_3 = pd.read_csv('./data/clip_preprocessing/clip_train_quantile0.75.csv')
test_3 = pd.read_csv('./data/clip_preprocessing/clip_test_quantile0.75.csv')
train_std = pd.read_csv('./data/clip_preprocessing/clip_train_std.csv')
test_std = pd.read_csv('./data/clip_preprocessing/clip_test_std.csv')
train_mid = pd.read_csv('./data/clip_preprocessing/clip_train_median.csv')
test_mid = pd.read_csv('./data/clip_preprocessing/clip_test_median.csv')
train_sem = pd.read_csv('./data/clip_preprocessing/clip_train_sem.csv')
test_sem = pd.read_csv('./data/clip_preprocessing/clip_test_sem.csv')

train_3.columns = [col+' q3' for col in train_3.columns]
test_3.columns = [col+' q3' for col in test_3.columns]
train_1.columns = [col+' q1' for col in train_1.columns]
test_1.columns = [col+' q1' for col in test_1.columns]
train_std.columns = [col+' std' for col in train_std.columns]
test_std.columns = [col+' std' for col in test_std.columns]
train_mid.columns = [col+' mid' for col in train_mid.columns]
test_mid.columns = [col+' mid' for col in test_mid.columns]
train_sem.columns = [col+' sem' for col in train_sem.columns]
test_sem.columns = [col+' sem' for col in test_sem.columns]

train = train.reset_index()
test  = test.reset_index()
train_3 = train_3.reset_index()
test_3 = test_3.reset_index()
train_1 = train_1.reset_index()
test_1 = test_1.reset_index()
train_std = train_std.reset_index()
test_std = test_std.reset_index()
train_mid = train_mid.reset_index()
test_mid = test_mid.reset_index()
train_sem = train_sem.reset_index()
test_sem = test_sem.reset_index()

In [20]:
train = pd.concat([train, train_1, train_3, train_std], axis=1)
test = pd.concat([test, test_1, test_3, test_std], axis=1)
train = train.reset_index(inplace=False)
test  = test.reset_index(inplace=False)
train = pd.concat([train, p_train], axis=1)
test = pd.concat([test, p_test], axis=1)
train = train.drop(['index', 'level_0'], axis=1)
test = test.drop(['index', 'level_0'], axis=1)

In [22]:
cat = CatBoostRegressor(verbose=100, early_stopping_rounds=50, task_type="GPU", random_seed=113)
model_train(train, test, cat)

Index(['DAT', '내부온도관측치', '내부습도관측치', 'co2관측치', 'ec관측치', '시간당분무량', '일간누적분무량',
       '시간당백색광량', '일간누적백색광량', '시간당적색광량', '일간누적적색광량', '시간당청색광량', '일간누적청색광량',
       '시간당총광량', '일간누적총광량', 'diff_temp', 'dark', 'water', '월간누적분무량',
       '월간누적백색광량', '월간누적청색광량', '월간누적적색광량', '내부온도관측치 q1', '내부습도관측치 q1',
       'co2관측치 q1', 'ec관측치 q1', '시간당분무량 q1', '시간당백색광량 q1', '시간당적색광량 q1',
       '시간당청색광량 q1', '시간당총광량 q1', '내부온도관측치 q3', '내부습도관측치 q3', 'co2관측치 q3',
       'ec관측치 q3', '시간당분무량 q3', '시간당백색광량 q3', '시간당적색광량 q3', '시간당청색광량 q3',
       '시간당총광량 q3', '내부온도관측치 std', '내부습도관측치 std', 'co2관측치 std', 'ec관측치 std',
       '시간당분무량 std', '시간당백색광량 std', '시간당적색광량 std', '시간당청색광량 std', '시간당총광량 std',
       '('일간누적백색광량', 1.0)', '('일간누적백색광량', 2.0)', '('일간누적백색광량', 3.0)',
       '('일간누적백색광량', 4.0)', '('일간누적분무량', 1.0)', '('일간누적분무량', 2.0)',
       '('일간누적분무량', 3.0)', '('일간누적분무량', 4.0)', '('일간누적적색광량', 1.0)',
       '('일간누적적색광량', 2.0)', '('일간누적적색광량', 3.0)', '('일간누적적색광량', 4.0)',
       '('일간누적청색광량', 1.0)', '('일간누적청색광량', 2.0)', '('일

In [23]:
df_imp = pd.DataFrame({'imp':cat.feature_importances_}, index = cat.feature_names_)
df_imp = df_imp[df_imp.imp > 0].sort_values('imp').copy()
df_imp.to_csv('feature_imp.csv', encoding='cp949')